# KNIT7
# Enable Precision Timing Capability on Nodes
This notebook downloads the FABRIC [Measurement Framework repository](https://github.com/fabric-testbed/MeasurementFramework) from github onto your jupyterhub instance and copies off an ansible playbook that would deploy the [linuxptp software package](https://linuxptp.sourceforge.net/) to enable PTP on your nodes. Sites used in the topology should be PTP capable.

## Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
import requests

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Get Slice Name

In [ ]:
slice_name=f"Slice for KNIT7 Precision Timing Tutorial"
mySlice = fablib.get_slice(name = slice_name)

## Observe Slice Attributes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

## Check if nodes in the slice are PTP Capable
Not all FABRIC sites have PTP Capability. The VMs use a virtual PTP device which is synced by the host. If the host is in sync with PTP, then the PTP time in the VMs will be accurate. If the SITE/HOST is not PTP Capable, the VM will receive time in the virtual PTP device that may not be true PTP. But that does not prevent the VMs from installing the software. So its up to you to decide whether to proceed or not in case the SITE/Node is not PTP Capable.

In [ ]:
nodes = mySlice.get_nodes()
for node in nodes:
    print (f"{node.get_name()} is hosted on {node.get_host()}")
    ad = fablib.get_site_advertisement(node.get_site())
    print (f"PTP Capable: { ad.flags.ptp}")

## Install and setup linuxptp package  on nodes
Download the Ansible role to configure and install the LinuxPTP software. We use code hosted in the FABRIC Measurement Framework github repository. For more details regarding the steps performed in the playbook, please refer to the repo at [https://github.com/fabric-testbed/MeasurementFramework/tree/main/instrumentize/ptp/ansible/roles/linuxptp](https://github.com/fabric-testbed/MeasurementFramework/tree/main/instrumentize/ptp/ansible/roles/linuxptp)

In [ ]:
pre_requisites = None

clone_instructions = f"""
cd /tmp/;git clone --filter=blob:none --no-checkout --depth 1 --sparse https://github.com/fabric-testbed/MeasurementFramework.git;
cd /tmp/MeasurementFramework;git sparse-checkout add instrumentize/ptp/ansible;git checkout;
"""

ansible_instructions = f"""
cd /tmp/MeasurementFramework/instrumentize/ptp/ansible;
ansible-playbook --connection=local --inventory 127.0.0.1, --limit 127.0.0.1 playbook_fabric_experiment_ptp.yml;
"""

#Create execute threads
execute_threads = {}

for node in nodes:
    if [ele for ele in ["rocky", "centos"] if (ele in node.get_image())]:
        pre_requisites = f"""
        sudo dnf -y install epel-release ; sudo dnf -y install ansible git; mkdir /tmp/ptp_ansible/;
        """
    elif [ele for ele in ["ubuntu", "debian"] if (ele in node.get_image())]:
        pre_requisites = f"""sudo apt-get update;sudo apt-get -y install ansible git;"""
    else:
        pre_requisites = None
    print (f"Installing PTP on {node.get_name()}")
    execute_threads[node] = node.execute_thread(\
                f"{pre_requisites}"\
                f"{clone_instructions}"\
                f"{ansible_instructions}",\
                output_file=f"/tmp/{node.get_name()}_ptpinstall.log"\
                )

    #Wait for results from threads
for node,thread in execute_threads.items():
    print(f"Waiting for result from node {node.get_name()}")
    stdout,stderr = thread.result()
    #print(f"stdout: {stdout}")
    #print(f"stderr: {stderr}")
    #node.execute(f"{pre_requisites}"\
    #             f"{ansible_instructions}"\
    #            )

print (f"Installation of PTP Completed\n\n")